In [1]:
import os as os
import requests as req
import zipfile as zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def data_directories(**kwargs):
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    Args:
        directories (list): Lista do caminho dos diretorios a serem criados.
        
    Return:
        Cria os diretorios para armazenamento dos dados.  
    """

    # Diretorios para armazenamento dos dados coletados
    kwargs['directories'] = ['..\\data\\01-collected'
                            ,'..\\data\\02-cleaned'
                            ,'..\\data\\03-organized'
                            ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Inicio da criacao dos diretorios para armazenamento dos dados.')
    
    # Para cada diretorio na lista de diretorios
    for directory in kwargs['directories']:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Fim da criacao dos diretorios para armazenamento dos dados.')

In [3]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo. Ao final do download, e verificado
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)
    
    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Log: Mensagem de URL encontrada e de inicio do download
            print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo csv 
        # na pasta e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])

In [7]:
def collect_data(**kwargs):
    """
    Coleta os dados para cada mes de referencia dentro das datas de inicio e fim informadas.
    
    Args:
        dt_ini_ref (date): Data inicial para coleta dos dados
        dt_fim_ref (date): Data final para coleta dos dados
    """
    
    # Define a data de referencia inicial
    kwargs['dt_ref'] = kwargs['dt_ini_ref']
    
    # Cria os diretorios para armazenamento dos dados
    data_directories(**kwargs)
    
    # Define o caminho completo para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    # Define a URL base de busca dos arquivos para download
    kwargs['download_url'] = 'http://www.portaltransparencia.gov.br/download-de-dados/cpgf/'
    
    # Define o nome do arquivo para download
    kwargs['file_name'] = '_CPGF'
    kwargs['file_ext'] = '.zip'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Aquisicao dos arquivos iniciada.')
       
    # Para cada mes de referencia dentro das datas de inicio e fim
    while kwargs['dt_ref'] <= kwargs['dt_fim_ref']:
        
        # Prepara as variaveis do mes de referencia
        kwargs['file_date_ref'] = kwargs['dt_ref'].strftime('%Y%m')
        kwargs['download_file'] = kwargs['file_date_ref'] + kwargs['file_name'] + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = kwargs['download_url'] + kwargs['file_date_ref']
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)
        
        # Atualiza o mes de refencia
        kwargs['dt_ref'] += relativedelta(months=1)

    # Log: Mensagem de finalizacao do processo 
    print(str(datetime.now()) + ': Aquisicao dos arquivos finalizada.')

In [8]:
def main():
    """
    Coleta os dados de Cartao de Pagamento do Governo Federal (CPGF)
    Origem: Portal da Transparencia
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Datas de Inicio e Fim (YYYY-MM-DD) 
    # Obs.: Dados disponiveis a partir de 2013-01-01
    kwargs['dt_ini_ref'] = datetime.strptime('2019-01-01', "%Y-%m-%d")
    kwargs['dt_fim_ref'] = datetime.strptime('2019-06-01', "%Y-%m-%d")
    
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2019-08-16 09:11:36.452934: Inicio da criacao dos diretorios para armazenamento dos dados.
2019-08-16 09:11:36.452934: Diretorio ..\data\01-collected ja existe.
2019-08-16 09:11:36.453929: Diretorio ..\data\02-cleaned ja existe.
2019-08-16 09:11:36.453929: Diretorio ..\data\03-organized ja existe.
2019-08-16 09:11:36.453929: Diretorio ..\data\04-standardized ja existe.
2019-08-16 09:11:36.453929: Fim da criacao dos diretorios para armazenamento dos dados.
2019-08-16 09:11:36.453929: Aquisicao dos arquivos iniciada.
2019-08-16 09:11:36.453929: Inicio do download do arquivo ..\data\01-collected\201901_CPGF.zip
2019-08-16 09:11:37.410851: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\201901_CPGF.zip
2019-08-16 09:11:37.493628: Descompactando o arquivo ..\data\01-collected\201901_CPGF.zip
2019-08-16 09:11:37.509587: Fim do download do arquivo ..\data\01-collected\201901_CPGF.zip
2019-08-16 09:11:37.510584: Inicio do download do arquivo ..\data\01-collected\201902_CPGF